In [1]:
using NBInclude
@nbinclude("Hofstadter Single Particle in Julia.ipynb")

Hofstadter_SP (generic function with 1 method)

In [2]:
using QuantumOptics

#https://juliapackages.com/p/einsum
using Einsum

# https://github.com/Jutho/TensorOperations.jl
using TensorOperations

using BenchmarkTools

In [3]:
function Hofstadter_SP_Op(Nx, Ny, alpha, Basis)
    
    H_SP = Hofstadter_SP(Nx,Ny,alpha, 0)
    
    N = Nx*Ny
    
    H = SparseOperator(Basis)

    for m in 1:N
        for n in 1:N
            H = H + H_SP[m,n] * transition(Basis, m, n)
        end
    end
    
    return H
end

Hofstadter_SP_Op (generic function with 1 method)

1) Hofstadter SP QoJulia energies are has to be equal to Hofstadter Single Particle energies.

In [4]:
Nx = 5; Ny = 5; N=Nx*Ny; q = 5; N_cut = 5
PN = 2
U = 2

2

In [5]:
using LinearAlgebra

basis = NLevelBasis(N)

NLevel(N=25)

In [6]:
H1 = Hofstadter_SP_Op(Nx, Ny, 1/q, basis)

Operator(dim=25x25)
  basis: NLevel(N=25)sparse([2, 5, 6, 21, 1, 3, 7, 22, 2, 4  …  22, 24, 4, 19, 23, 25, 5, 20, 21, 24], [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 8, 9, 9, 9, 9, 10, 10, 10, 10, 11, 11, 11, 11, 12, 12, 12, 12, 13, 13, 13, 13, 14, 14, 14, 14, 15, 15, 15, 15, 16, 16, 16, 16, 17, 17, 17, 17, 18, 18, 18, 18, 19, 19, 19, 19, 20, 20, 20, 20, 21, 21, 21, 21, 22, 22, 22, 22, 23, 23, 23, 23, 24, 24, 24, 24, 25, 25, 25, 25], ComplexF64[-1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, -0.30901699437494745 - 0.9510565162951536im, -0.30901699437494745 + 0.9510565162951536im, -1.0 + 0.0im, -1.0 + 0.0im  …  -1.0 + 0.0im, -1.0 + 0.0im, 0.8090169943749476 + 0.587785252292473im, 0.8090169943749476 - 0.587785252292473im, -1.0 + 0.0im, -1.0 + 0.0im, -0.30901699437494723 + 0.9510565162951536im, -0.30901699437494723 - 0.9510565162951536im, -1.0 + 0.0im, -1.0 + 0.0im], 25, 25)

In [7]:
eigenenergies(dense(H1))

25-element Vector{Float64}:
 -2.9664479891433726
 -2.9664479891433726
 -2.966447989143372
 -2.966447989143372
 -2.966447989143372
 -0.9021130325903075
 -0.9021130325903074
 -0.9021130325903071
 -0.9021130325903066
 -0.9021130325903048
 -0.38196601125010654
 -0.3819660112501064
 -0.381966011250106
 -0.3819660112501059
 -0.38196601125010454
  1.3484140003934764
  1.348414000393477
  1.3484140003934773
  1.348414000393479
  1.3484140003934793
  2.902113032590303
  2.9021130325903046
  2.902113032590308
  2.902113032590309
  2.902113032590309

In [8]:
eigenenergies(dense(H1)) == eigvals(Hofstadter_SP(Nx,Ny,1/q,0))

true

In [9]:
function get_sub_states(SP_Op, N_cut)
    
    E0, states0 = eigenstates(dense(SP_Op))
    states = states0[1:N_cut]
    
    return states
end

get_sub_states (generic function with 1 method)

In [10]:
sub_states = get_sub_states(H1, N_cut);

In [11]:
function get_Projector(states, Basis)
    
    b_sub = SubspaceBasis(Basis,states)
    P = projector(b_sub, Basis)
    Pt = dagger(P)
    
    return b_sub, P, Pt
end

get_Projector (generic function with 1 method)

In [12]:
b_sub, P, Pt = get_Projector(sub_states, basis);

In [13]:
function Subspace_Op(SP_Op, P)
    return P*SP_Op*Pt
end

Subspace_Op (generic function with 1 method)

In [14]:
H1_sub = Subspace_Op(H1, P)

Operator(dim=5x5)
  basis: Subspace(superbasis=NLevel(N=25), states:5)
 -2.96645+1.0e-17im   1.1e-16-6.0e-17im  …  -7.0e-17+3.0e-17im
  1.7e-16+0.0im      -2.96645-1.1e-16im     -8.0e-17-1.0e-16im
 -3.0e-17+8.0e-17im       0.0+1.4e-16im      2.8e-16-2.2e-16im
 -6.0e-17-6.0e-17im   2.2e-16+0.0im          3.6e-16+2.2e-16im
 -4.0e-17-4.0e-17im  -6.0e-17+7.0e-17im     -2.96645+1.1e-16im

In [15]:
num_sub_list = []
for m in 1:N
    NM = transition(basis, m, m)
    NMP = Subspace_Op(NM, P)
    push!(num_sub_list, NMP)
end

In [16]:
states_mb = bosonstates(b_sub, PN) 
basis_mb = ManyBodyBasis(b_sub, states_mb)

ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=25), states:5), states:15)

In [17]:
function get_MB_Op(MB_Basis, Basis, Op)
    
    Op_MB = SparseOperator(MB_Basis)
    for i in 1:length(Basis)
        for j in 1:length(Basis)
            Op_MB = Op_MB + Op.data[i,j] * transition(MB_Basis, i, j)
        end
    end
    
    return Op_MB
end

H1_MB = get_MB_Op(basis_mb, b_sub, H1_sub)

Operator(dim=15x15)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=25), states:5), states:15)sparse([1, 2, 3, 4, 5, 1, 2, 3, 4, 5  …  11, 12, 13, 14, 15, 5, 9, 12, 14, 15], [1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15], ComplexF64[-5.932895978286744 + 2.0e-17im, 2.4e-16 + 0.0im, -4.0e-17 + 1.2e-16im, -8.0e-17 - 8.0e-17im, -6.0e-17 - 6.0e-17im, 1.6e-16 - 8.0e-17im, -5.932895978286743 - 1.0e-16im, 0.0 + 1.4e-16im, 2.2e-16 + 0.0im, -6.0e-17 + 7.0e-17im  …  2.8e-16 - 2.2e-16im, 1.17e-15 - 2.2e-16im, 5.1e-16 + 3.1e-16im, -5.932895978286745 + 2.2e-16im, 1.2e-16 + 3.1e-16im, -1.0e-16 + 4.0e-17im, -1.1e-16 - 1.4e-16im, 3.9e-16 - 3.1e-16im, 5.1e-

In [18]:
# H1_mb = manybodyoperator(basis_mb, H1_sub)

In [19]:
num_mb_list = []
for m in 1:N
    NMP = get_MB_Op(basis_mb, b_sub, num_sub_list[m])
    push!(num_mb_list, NMP)
end

In [20]:
function Int_Op(MB_Num_Op_List, MB_Basis, U)

    IT = SparseOperator(MB_Basis)
    
    for m in 1:N
        IT = IT + U/2 * ( MB_Num_Op_List[m]*MB_Num_Op_List[m] - MB_Num_Op_List[m] )
    end
    
    return IT
end

H_Int_MB = Int_Op(num_mb_list, basis_mb, U)

Operator(dim=15x15)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=25), states:5), states:15)sparse([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15], ComplexF64[-1.3961777052703093 + 0.0im, 0.005638

In [30]:
#@btime begin
basis2 = basis ⊗ basis

# interaction : at_i at_i a_i a_i = at_i a_i at_i a_i - at_i a_i = n_i n_i - n_i
    
Vint = SparseOperator(basis2)

for n in 1:N
    Vint += U/2*transition(basis,n,n)⊗transition(basis,n,n)
end

Vint_sub = (P⊗P)*Vint*(Pt⊗Pt)

Vint_mb = manybodyoperator(basis_mb, Vint_sub)
#end

Operator(dim=15x15)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=25), states:5), states:15)
     0.203822-0.0im          …   -0.00262863+0.00113256im
   0.00563848-0.000335968im         0.013185-0.0014774im
  -0.00364264-0.00225113im       0.000190761-0.00158488im
 -0.000990291+0.00104066im      -0.000642365-0.00587854im
  -0.00023807+0.000969258im         0.002896+0.00241287im
    0.0241578+0.0043468im    …   -0.00183871+0.00186099im
 -0.000101367+0.00645821im        0.00206597-0.00590343im
    0.0209507-0.0088019im        -0.00522229+0.0252256im
 -0.000126074-0.00693268im        0.00938242+0.0126401im
   0.00960656+0.00889825im         0.0150597+0.000322088im
    0.0154469-0.00144025im   …    0.00876078-0.0258647im
  -0.00333547+0.00397896im        -0.0132376+0.0116158im
  -0.00392264-0.017573im         0.000297104-0.00116768im
  -0.00356431+0.00114511im       -0.00303801-0.0116332im
  -0.00262863-0.00113256im          0.195833+0.0im

In [33]:
#@btime begin
P1 = P.data
P1t = Pt.data;

@einsum P4[k,l,m,n] := P1[k,i] * P1[l,i] * P1t[i,m] * P1t[i,n]

b2_sub = b_sub ⊗ b_sub

P4re = reshape(P4, N_cut^2, N_cut^2)

P2_op = Operator(b2_sub,U/2*P4re)
    
Vint_mb_einsum = manybodyoperator(basis_mb, P2_op)
#end

Operator(dim=15x15)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=25), states:5), states:15)
     0.203822-0.0im          …   -0.00262863+0.00113256im
   0.00563848-0.000335968im         0.013185-0.0014774im
  -0.00364264-0.00225113im       0.000190761-0.00158488im
 -0.000990291+0.00104066im      -0.000642365-0.00587854im
  -0.00023807+0.000969258im         0.002896+0.00241287im
    0.0241578+0.0043468im    …   -0.00183871+0.00186099im
 -0.000101367+0.00645821im        0.00206597-0.00590343im
    0.0209507-0.0088019im        -0.00522229+0.0252256im
 -0.000126074-0.00693268im        0.00938242+0.0126401im
   0.00960656+0.00889825im         0.0150597+0.000322088im
    0.0154469-0.00144025im   …    0.00876078-0.0258647im
  -0.00333547+0.00397896im        -0.0132376+0.0116158im
  -0.00392264-0.017573im         0.000297104-0.00116768im
  -0.00356431+0.00114511im       -0.00303801-0.0116332im
  -0.00262863-0.00113256im          0.195833+0.0im

In [34]:
isapprox( P2_op, Vint_sub)

true

In [35]:
isapprox( Vint_mb , Vint_mb_einsum )

true

In [36]:
H_MB = H1_MB + Vint_mb_einsum
H_mb = H1_MB + Vint_mb

Operator(dim=15x15)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=25), states:5), states:15)
     -5.72907+2.0e-17im      …   -0.00262863+0.00113256im
   0.00563848-0.000335968im         0.013185-0.0014774im
  -0.00364264-0.00225113im       0.000190761-0.00158488im
 -0.000990291+0.00104066im      -0.000642365-0.00587854im
  -0.00023807+0.000969258im         0.002896+0.00241287im
    0.0241578+0.0043468im    …   -0.00183871+0.00186099im
 -0.000101367+0.00645821im        0.00206597-0.00590343im
    0.0209507-0.0088019im        -0.00522229+0.0252256im
 -0.000126074-0.00693268im        0.00938242+0.0126401im
   0.00960656+0.00889825im         0.0150597+0.000322088im
    0.0154469-0.00144025im   …    0.00876078-0.0258647im
  -0.00333547+0.00397896im        -0.0132376+0.0116158im
  -0.00392264-0.017573im         0.000297104-0.00116768im
  -0.00356431+0.00114511im       -0.00303801-0.0116332im
  -0.00262863-0.00113256im          -5.73706+2.2e-16im

2) If U<<Band-Gap, Sub-Space Hofstadter Finite-U energies converges at Hofstadter Finite-U energies.
3) When all SP states projected to the new sub-space, Sub-Space Hofstadter Finite-U energies have to equal to Hofstadter Finite-U energies.

In [37]:
@nbinclude("Hofstadter MB in Julia.ipynb"; regex=r"#.*executeme")

Hofstadter_Finite_U (generic function with 1 method)

In [38]:
# using Plots

E1 = eigenenergies(dense((H_MB+dagger(H_MB))/2))

E_mb = eigenenergies(dense((H_mb+dagger(H_mb))/2))


E2 = eigenenergies(dense(Hofstadter_Finite_U(Nx, Ny, 1/q, PN, U)))

# print(E1-E2[1:length(E1)])

# plot(1:length(E1),E1,seriestype=:scatter,markershape=:star5, markersize=6, label="Sub-Space")
# plot!(1:length(E2),E2,seriestype=:scatter,label="Real-Space",legend=:topleft)
# xlabel!("n");ylabel!("E")

print(E1,'\n', E_mb,'\n',E2[1:10])

[-5.930026862496533, -5.930026862496531, -5.930026862496528, -5.930026862496527, -5.930026862496525, -5.772090091593084, -5.772090091593081, -5.772090091593079, -5.772090091593078, -5.772090091593076, -5.696570980770627, -5.696570980770624, -5.6965709807706215, -5.696570980770621, -5.696570980770616]
[-5.930026862496533, -5.930026862496531, -5.93002686249653, -5.930026862496528, -5.930026862496526, -5.772090091593085, -5.772090091593083, -5.772090091593081, -5.772090091593077, -5.772090091593075, -5.696570980770626, -5.696570980770625, -5.696570980770624, -5.69657098077062, -5.696570980770619]
[-5.9308125874565, -5.930812587456478, -5.930812587456468, -5.9308125874564555, -5.930812587456455, -5.817854200657324, -5.817854200657313, -5.81785420065731, -5.817854200657299, -5.8178542006572815]

In [39]:
E_mb

15-element Vector{Float64}:
 -5.930026862496533
 -5.930026862496531
 -5.93002686249653
 -5.930026862496528
 -5.930026862496526
 -5.772090091593085
 -5.772090091593083
 -5.772090091593081
 -5.772090091593077
 -5.772090091593075
 -5.696570980770626
 -5.696570980770625
 -5.696570980770624
 -5.69657098077062
 -5.696570980770619